In [3]:
import pandas as pd
import json
import numpy as np
from bs4 import BeautifulSoup
import math


In [9]:
assist_df = pd.read_csv('/Users/kiakarbasi/pykt_emb_chef/pykt-toolkit-pt_emb/embeddings/data/assist2009/full_dataset/skill_builder_data_corrected_collapsed.csv', encoding = "ISO-8859-1", low_memory=False)
pb_df = pd.read_csv('/Users/kiakarbasi/pykt_emb_chef/pykt-toolkit-pt_emb/embeddings/data/problem_bodies/ProblemBodies_23.csv', low_memory=False)
subset_df_assist = pd.read_csv('/Users/kiakarbasi/pykt_emb_chef/pykt-toolkit-pt_emb/embeddings/data/assist2009/pb_subset/skill_builder_data_corrected_collapsed_pb_subset.csv', low_memory=False)




In [30]:
temp_df = pb_df[pb_df['curriculum']== 'Skill Builders']
temp_df[temp_df['grade_or_subject']== 'High School: Geometry']['problem_type'].unique()
temp1_df = temp_df[temp_df['grade_or_subject']== 'High School: Geometry']
temp1_df[temp1_df['problem_type']=='old_algebra']


temp2_df = temp_df[temp_df['grade_or_subject'].isin(['High School: Number and Quantity', 'High School: Algebra ', 
       'High School: Functions', 
       'High School: Statistics and Probability', 'High School: Geometry'])]


temp2_df ['unit'].unique()

x = temp2_df ['problem_set_name'].unique()

In [35]:
x
# x.shape

array(['(N.VM.C.8) Matrix Addition Skill Builder EX',
       '(N.VM.C.8) Matrix Rows and Columns Skill Builder',
       '(A.SSE.A.1a) Interpreting Parts of an Expression Skill Builder',
       '(N.VM.C.8) Matrix Multiplication Skill Builder',
       '(F.BF.A.1c) Composition of Functions Skill Builder',
       '(F.BF.A.2) Geometric and Arithmetic Sequences Skill Builder',
       '(F.BF.B.5) Logarithm Base Change Skill Builder',
       '(F.BF.B.4) Finding Inverse Functions Skill Builder',
       '(F.LE.A.4a) Solving ab^ct = d Skill Builder',
       '(F.BF.B.5) Basic Logarithm Manipulation Skill Builder',
       '(A.SSE.A.1a) Standard Form of a Polynomial Skill Builder',
       '(N.CN.A.2) Complex Numbers Skill Builder',
       '(F.TF.A.1) Converting Radians to Degrees Skill Builder',
       '(F.IF.B.4) Interpreting Graphs of Functions Skill Builder',
       '(F.BF.B.5) Intermediate Logarithm Manipulation Skill Builder',
       '(SS.MD.B.5) Finding Expected Value Skill Builder EX',
      

In [5]:
questions = assist_df['problem_id'].unique()
assist_df['skill_id'].unique()

skill_to_problems = {}
for skill_id in assist_df['skill_id'].unique():
  skill_to_problems[skill_id] = assist_df[assist_df['skill_id'] == skill_id]['problem_id'].unique()


In [6]:
with open('/Users/kiakarbasi/pykt_emb_chef/pykt-toolkit-pt_emb/embeddings/data/assist2009/pb_subset/keyid2idx_assist2009_pb_subset.json', 'r') as f:
  keyid2idx_pb_subset = json.load(f)

In [7]:
problem_ids = list(keyid2idx_pb_subset['questions'].keys())
problem_ids_int = [int(x) for x in problem_ids]
subset_df_html_selected = pb_df[pb_df['problem_id'].isin(problem_ids_int)]
subset_df_html = subset_df_html_selected[['problem_id','problem_body']]

In [7]:
from bs4 import BeautifulSoup

def remove_tags(html):
    soup = BeautifulSoup(html, "html.parser")

    # Find and replace <img> tags with their 'alt' attribute
    for img in soup.find_all('img'):
        alt_text = img.get('alt', '')  # default to empty string if 'alt' is None
        if alt_text:
            # Create a new text node
            new_text = soup.new_string(" " + alt_text + " ")
            img.replace_with(new_text)
        else:
            # Remove the image if no alt text
            img.decompose()

    # Remove all script and style elements
    for tag in soup(['script', 'style']):
        tag.decompose()

    # Extract the text, cleaning up any excessive whitespace
    clean_text = ' '.join(soup.stripped_strings)
    return clean_text

# Example usage with a DataFrame column
# subset_df['problem_body'] = subset_df['problem_body'].apply(remove_tags)




#pass pb_df_subset['problem_body'] column into function remove_tags
subset_df_html['problem_body'] = subset_df_html['problem_body'].apply(remove_tags)

/Users/kiakarbasi/anaconda3/envs/kiapykt/lib/python3.7/site-packages/ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [9]:
subset_df_html.to_csv('//Users/kiakarbasi/pykt_emb_chef/pykt-toolkit-pt_emb/embeddings/data/problem_bodies/pb_assist2009_subset_preprocessed.csv', index=False)

In [8]:
subset_df_html

,problem_id,problem_body
2190,119299.0,<p>First lets understand that the problem is a...
2654,97581.0,<p>We now have that</p>\n<p>4 + 10x = -16&nbsp...
2933,76339.0,"<p>The <span style=""color: #ff0000;"">height1</..."
2941,89068.0,<p>What is the sum of the angles inside an <st...
3660,76309.0,<p>Now since we know the trapezoid is in fact ...
...,...,...
472553,90055.0,"<p>&nbsp;What is <img class=""Wirisformula"" sty..."
472614,90238.0,"<p>What is <img class=""Wirisformula"" style=""ma..."
472619,90230.0,"<p>What is&nbsp;<img class=""Wirisformula"" styl..."
472622,90174.0,"<p>What is <img class=""Wirisformula"" style=""ma..."


In [13]:
subset_df_assist['problem_id'].unique().shape

(7836,)

In [14]:
subset_df_html['problem_id'].unique().shape

(6035,)

In [16]:
problem_ids
len(problem_ids)

6035

In [17]:
concept_ids = list(keyid2idx_pb_subset['concepts'].keys())
len(concept_ids)

76

In [19]:
subset_df_assist['skill_id'].unique().shape

(78,)

In [18]:
subset_df_assist


,Unnamed: 0,Unnamed: 0.1,order_id,assignment_id,user_id,assistment_id,problem_id,original,correct,attempt_count,...,hint_count,hint_total,overlap_time,template_id,answer_id,answer_text,first_action,bottom_hint,opportunity,opportunity_original
0,3957,3958,21617623,263599,14,53412,93383,1,0,1,...,2,2,41131,52570,NaN,NaN,1,1.0,1,1.0
1,3958,3959,21617632,263599,14,53436,93407,1,1,1,...,0,2,29123,52570,NaN,82.8,0,NaN,2,2.0
2,3959,3960,21617641,263599,14,53429,93400,1,0,1,...,2,2,19905,52570,NaN,NaN,1,1.0,3,3.0
3,3960,3961,21617650,263599,14,53448,93419,1,0,1,...,2,2,22600,52570,NaN,NaN,1,1.0,4,4.0
4,3961,3962,21617659,263599,14,53449,93420,1,0,1,...,2,2,19704,52570,NaN,NaN,1,1.0,5,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110444,346855,401752,38214014,291495,96299,57830,108976,0,1,1,...,0,2,38234,55692,200260.0,NaN,0,NaN,54,NaN
110445,346856,401753,38214016,291495,96299,57843,109015,0,1,1,...,0,0,6500,55693,200299.0,NaN,0,NaN,55,NaN
110446,346857,401754,38214195,291503,96299,34577,54060,0,1,1,...,0,3,18344,30677,NaN,0.8,0,NaN,56,NaN
110447,346858,401755,38214196,291503,96299,34577,54061,0,1,1,...,0,2,12922,30677,NaN,-6,0,NaN,57,NaN


In [3]:
import openai


ImportError: cannot import name 'deprecated' from 'typing_extensions' (/Users/kiakarbasi/anaconda3/envs/kiapykt/lib/python3.7/site-packages/typing_extensions.py)